In [4]:
# car_chatbot_colab_demo.py
"""
Colab‑friendly demo chatbot that can:
1. Parse intents & extract entities with spaCy
2. Retrieve answers from a simulated knowledge base (features, troubleshooting, manual)
3. Handle multi‑turn dialogue with simple context tracking
4. Gauge sentiment using NLTK VADER; if frustration detected, offer escalation

Usage in Google Colab
--------------------
```python
# 1️⃣  Install deps (run once):
!pip -q install spacy nltk pandas
!python -m spacy download en_core_web_sm

# 2️⃣  Load the chatbot:
%run car_chatbot_colab_demo.py

# 3️⃣  Start chatting:
chatbot = CarChatbot()
chatbot.chat()
```
If you ever see a missing‑lexicon error, rerun the install cell or just rely on the automatic downloader built into the script.
"""

import re
import json
from pathlib import Path
import subprocess, sys

import pandas as pd
import spacy
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# ---------- 1. Init NLP components ----------
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    # Grab the model on the fly if the user skipped the install step.
    subprocess.run([sys.executable, "-m", "spacy", "download", "en_core_web_sm"], check=True)
    nlp = spacy.load("en_core_web_sm")

# Ensure the VADER lexicon is present for sentiment analysis
try:
    sid = SentimentIntensityAnalyzer()
except LookupError:
    nltk.download("vader_lexicon")
    sid = SentimentIntensityAnalyzer()

# ---------- 2. Simulated knowledge base ----------
FEATURES_KB = {
    # Original
    "sunroof": "The power sunroof can tilt or slide open using the overhead switch.",
    "adaptive_cruise_control": "ACC maintains set speed and distance; press the steering‑wheel ACC button to activate and use +/- to adjust.",
    "lane_assist": "Lane‑Keeping Assist steers gently to keep the vehicle centered when lane markers are visible.",
    "heated_seats": "Press the seat‑icon button on the climate panel to cycle through high, medium, low, and off.",
    "parking_sensors": "Sensors emit beeps that increase in frequency as you near an obstacle – active in REVERSE or below 6 mph.",
    "apple_carplay": "Connect via USB (or wireless if equipped) and accept the CarPlay prompt on both car and phone.",
    "android_auto": "Plug in with a quality USB‑C cable or pair wirelessly, then follow on‑screen steps to launch Android Auto.",
    "blind_spot_monitor": "An amber icon lights up on the side mirror when another vehicle is in your blind spot.",
    "automatic_headlights": "Leave the stalk in AUTO; the sensor switches between DRL and low‑beam as ambient light changes.",
    "remote_start": "Press lock once, then hold the remote‑start button for 3 seconds; lights flash when the engine starts.",
    "head_up_display": "The HUD projects speed and navigation onto the windshield; adjust height in Settings › Display › HUD.",
    "360_camera": "Bird‑eye view combines images from four cameras to aid parking; press the CAMERA button to activate.",
    "wireless_charging_pad": "Place a Qi‑enabled phone on the pad; the LED turns orange while charging, green when full.",
    "ambient_lighting": "Change cabin accent colors in Settings › Lighting. Brightness syncs with headlight state.",
    "driver_memory_seat": "Hold SET then press 1 or 2 after adjusting seat, mirrors, and HUD to store your profile.",
    "power_tailgate": "Press the dash button or key‑fob twice to open/close the tailgate electrically.",
    "hands_free_liftgate": "With key in pocket, kick under the rear bumper to open the liftgate hands‑free.",
    "rain_sensing_wipers": "Wipers auto‑activate when moisture is detected; adjust sensitivity with the stalk ring.",
    "auto_dimming_mirror": "Mirror darkens automatically against glare; toggle the rear switch to disable.",
    "traffic_sign_recognition": "Front camera reads speed‑limit signs and shows them in the cluster display.",
    "automatic_emergency_braking": "AEB brakes automatically to avoid collisions below ~30 mph; keep radar area clear of debris.",
    "forward_collision_warning": "FCW gives audible & visual alerts if closing too quickly on a vehicle ahead.",
    "rear_cross_traffic_alert": "While reversing, RCTA warns of vehicles approaching from the sides.",
    "hill_descent_control": "HDC maintains low speed on steep downhill grades; press the HDC button while in 4‑Low.",
    "hill_start_assist": "HSA holds brake pressure for 2 seconds on hills to prevent rollback when starting.",
    "drive_mode_selector": "Rotate the console dial to choose Eco, Comfort, Sport, Snow, or Off‑Road modes.",
    "sport_mode": "Sport mode sharpens throttle response and holds gears longer for spirited driving.",
    "eco_mode": "Eco mode softens acceleration and optimizes A/C to maximise fuel economy.",
    "air_ionizer": "Cabin air ionizer reduces particulates and odors when RECIRC is active.",
    "rear_seat_reminder": "An alert sounds if rear doors opened at start of trip but not reopened when you stop.",
    "child_safety_lock": "Activate electronic child locks via Settings › Doors; an icon confirms engagement.",
    "stop_start_system": "Engine auto‑stops at traffic lights; depress the brake firmly. Disable via the A‑OFF button.",
    "adaptive_headlights": "Headlights swivel with steering angle to illuminate curves.",
    "matrix_led_headlights": "Matrix LEDs mask out oncoming traffic while maintaining high beam elsewhere.",
    "night_vision_assist": "Infra‑red camera highlights pedestrians/animals beyond headlight range on the cluster screen.",
    "adaptive_damping": "Electronic dampers adjust firmness every 10 ms based on road & drive mode.",
    "air_suspension": "Air suspension raises for off‑road (+40 mm) or lowers at highway speed (‑15 mm).",
    "active_noise_cancellation": "Cabin microphones detect road noise; speakers emit counter‑phase sound to cancel it.",
    "gesture_control": "Wave right‑hand gestures near the console sensor to change tracks or adjust volume.",
    "voice_control": "Say 'Hey Car' followed by a command to set navigation, climate, or seat heat.",
    "wifi_hotspot": "Enable Wi‑Fi Hotspot in Connectivity menu; supports up to 8 devices on 4G LTE.",
    "over_the_air_updates": "Vehicle software downloads overnight via Wi‑Fi; schedule install under Settings › Software.",
    "digital_instrument_cluster": "12.3‑inch cluster offers Classic, Sport, and Map‑View layouts selectable with the ◄► buttons.",
    "remote_parking": "Use the key‑fob arrows to move the car straight in/out of tight spaces remotely.",
    "valet_mode": "Valet Mode limits speed to 55 mph and hides personal data; set a PIN to activate.",
    "battery_preconditioning": "In EV mode, schedule pre‑conditioning so the battery warms before fast‑charging.",
    "plug_in_charge_port": "The charge port is on the driver‑side rear; press the door to open or use the key.",
    "regen_braking_strength": "Tap the steering paddles to cycle regenerative braking between Levels 0‑3.",
    "supercharger_locator": "The nav automatically suggests Superchargers on long trips; sites show power availability.",
    "summon_feature": "From the app, use Smart Summon to have the car drive to you within line‑of‑sight.",
}

TROUBLE_KB = {
    "engine_wont_start": "Check that the key fob battery isn't depleted, the gear selector is in PARK, and the brake pedal is fully depressed while pressing START.",
    "bluetooth_not_connecting": "Delete the car from your phone's Bluetooth list, then redo the pairing process. Ensure no more than 5 devices are stored in the infotainment system.",
    "ac_not_cold": "Verify that the A/C button is illuminated. If so, inspect cabin air filter (behind the glovebox) for clogging. Low refrigerant may require service.",
}
MANUAL_KB = {
    # Original 10
    "tire_pressure": "Cold tire pressure: 35 psi front / 33 psi rear.",
    "oil_type": "Use 0W‑20 synthetic oil (API SP). Capacity 4.7 qt with filter.",
    "fuse_box_location": "Cabin fuse panel is behind driver‑lower dash. Spare fuses included.",
    "maintenance_schedule": "Oil 10 k mi; cabin filter 15 k mi; brake fluid 24 mo; coolant 60 k mi.",
    "wiper_blade_size": "26 in driver, 18 in passenger, 12 in rear.",
    "coolant_type": "HOAT 50/50 premix ethylene‑glycol. Capacity 8.2 L.",
    "spark_plug_gap": "0.044 in (1.1 mm) iridium plugs.",
    "battery_location": "12 V battery: under‑hood, passenger‑side rear corner.",
    "towing_capacity": "Max braked: 3 500 lb (with trailer brakes).",
    "fuel_tank_capacity": "Fuel tank holds 14.5 US gallons including 2 gal reserve.",
    "brake_pad_min_thickness": "Replace brake pads when friction material is ≤3 mm.",
    "transmission_fluid_type": "Use ATF WS; service interval 60 k mi.",
    "recommended_octane": "Use unleaded gasoline 91 AKI (95 RON) or higher for optimal performance.",
    "engine_air_filter_part": "Engine air filter element: OEM #17801‑0H050.",
    "cabin_filter_part": "Cabin filter: OEM #87139‑02020 activated‑carbon type.",
    "lug_nut_torque": "Torque alloy‑wheel lug nuts to 85 lb‑ft (115 N·m).",
    "serpentine_belt_routing": "See label under hood for belt path; tensioner located passenger‑side.",
    "vin_location": "VIN is stamped on driver‑side dash, visible through windshield, and on door‑jamb label.",
    "jack_points": "Use reinforced pinch‑welds behind front wheels and ahead of rear wheels.",
    "child_seat_anchors": "Lower anchors located at the seat bight; top‑tethers on rear shelf.",
    "tire_rotation_pattern": "For FWD, move front to rear same side; rear to front opposite sides every 5 k mi.",
    "wheel_alignment_spec": "Front camber −0°45′ ± 0°30′, toe 0.00 ± 0.05 in.",
    "idle_rpm": "Normal warm‑idle speed: 650 ± 50 RPM.",
    "headlight_bulb_type": "Low‑beam: H11 55 W halogen; High‑beam: 9005 60 W.",
    "obd_port_location": "OBD‑II connector is under the steering column, left of the brake pedal.",
    "reset_service_light": "Turn ignition to ON, hold trip button, power OFF, keep holding, power ON until light resets.",
}


# Map of simple regex patterns to intents (could be replaced by ML classifier)
INTENT_PATTERNS = {
    "feature": re.compile(r"(how|what|tell me).*?(sunroof|lane assist|adaptive cruise)", re.I),
    "troubleshoot": re.compile(r"(won't start|not working|not connecting|error|problem|issue)", re.I),
    "manual": re.compile(r"(oil|tire|pressure|fuse|capacity|spec|manual)", re.I),
}

# ---------- 3. Core chatbot class ----------
class CarChatbot:
    def __init__(self):
        self.context = {}

    # --- Intent & entity detection ---
    def _detect_intent(self, text):
        """Very lightweight intent detection via regex rules + spaCy entities."""
        for intent, pattern in INTENT_PATTERNS.items():
            if pattern.search(text):
                return intent
        return "unknown"

    def _extract_entities(self, text, intent):
        doc = nlp(text)
        if intent == "feature":
            for key in FEATURES_KB:
                if key.replace("_", " ") in text.lower():
                    return {"feature": key}
        elif intent == "troubleshoot":
            for key in TROUBLE_KB:
                if any(word in text.lower() for word in key.split("_")):
                    return {"issue": key}
        elif intent == "manual":
            for key in MANUAL_KB:
                if any(word in text.lower() for word in key.split("_")):
                    return {"topic": key}
        return {}

    # --- Sentiment check ---
    def _is_frustrated(self, text):
        score = sid.polarity_scores(text)
        return score["compound"] < -0.5

    # --- Response generator ---
    def _generate_response(self, intent, entities, text):
        if intent == "feature" and entities.get("feature"):
            return FEATURES_KB[entities["feature"]]
        if intent == "troubleshoot" and entities.get("issue"):
            return TROUBLE_KB[entities["issue"]]
        if intent == "manual" and entities.get("topic"):
            return MANUAL_KB[entities["topic"]]
        return "Sorry, I didn't quite get that. Could you rephrase or specify what you're looking for?"

    # --- Chat loop for Colab ---
    def chat(self):
        print("  CarBot ready! Ask me about features, troubleshooting, or manual info. Type 'quit' to end.\n")
        while True:
            user = input("You: ")
            if user.lower().strip() in {"quit", "exit"}:
                print("CarBot: Bye! Drive safe.")
                break
            intent = self._detect_intent(user)
            entities = self._extract_entities(user, intent)
            response = self._generate_response(intent, entities, user)

            # Sentiment escalation
            if self._is_frustrated(user):
                response += "\nI sense some frustration. Would you like me to connect you to a human support agent?"
            print(f"CarBot: {response}\n")

# Allow module execution: running this file launches immediate chat (handy for Colab %run)
if __name__ == "__main__":
    CarChatbot().chat()


  CarBot ready! Ask me about features, troubleshooting, or manual info. Type 'quit' to end.

You: brake pads
CarBot: Sorry, I didn't quite get that. Could you rephrase or specify what you're looking for?

You: eco_mode
CarBot: Sorry, I didn't quite get that. Could you rephrase or specify what you're looking for?

You: blutooth isn't connecting to my phone - any idea ??
CarBot: Sorry, I didn't quite get that. Could you rephrase or specify what you're looking for?

You: My A/C isn’t blowing cold air
CarBot: Sorry, I didn't quite get that. Could you rephrase or specify what you're looking for?



KeyboardInterrupt: Interrupted by user

CarBot ready! Ask me about features, troubleshooting, or manual info. Type 'quit' to exit.

You: How do I switch on the adaptive headlights?
CarBot: Headlights swivel with steering angle to illuminate curves.

You: My bluetooth keeps dropping connection.
CarBot: Sorry, I didn't quite get that. Could you rephrase or specify what you're looking for?

You: A/C is blowing warm air
CarBot: Sorry, I didn't quite get that. Could you rephrase or specify what you're looking for?

You: A/C is blowing warm air
CarBot: Sorry, I didn't quite get that. Could you rephrase or specify what you're looking for?



KeyboardInterrupt: Interrupted by user

In [11]:
# car_chatbot_colab_demo_v2.py  –  Stable v2.2 (May-2025)
"""
Colab quick-start
─────────────────
!pip -q install spacy nltk rapidfuzz openai numpy
!python -m spacy download en_core_web_sm

# (Optional) add your key once per runtime:
import os; os.environ["OPENAI_API_KEY"] = "sk-..."

%run car_chatbot_colab_demo_v2.py
bot = CarChatbot(); bot.chat()
"""

from __future__ import annotations
import os, sys, subprocess, unicodedata, re, math
from pathlib import Path

# ──────────────────────────────────────────────────────────────────────────────
# 1. Auto-install / import third-party deps
# ──────────────────────────────────────────────────────────────────────────────
def _ensure(pkg: str, import_name: str | None = None):
    import importlib
    try:
        return importlib.import_module(import_name or pkg)
    except ModuleNotFoundError:
        subprocess.run([sys.executable, "-m", "pip", "install", pkg, "--quiet"], check=True)
        return importlib.import_module(import_name or pkg)

spacy     = _ensure("spacy")
nltk      = _ensure("nltk")
rapidfuzz = _ensure("rapidfuzz")
numpy     = _ensure("numpy", "numpy")
openai    = _ensure("openai")
from rapidfuzz import fuzz
np = numpy

# ──────────────────────────────────────────────────────────────────────────────
# 2. Local models / sentiment
# ──────────────────────────────────────────────────────────────────────────────
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    subprocess.run([sys.executable, "-m", "spacy", "download", "en_core_web_sm"], check=True)
    nlp = spacy.load("en_core_web_sm")

from nltk.sentiment.vader import SentimentIntensityAnalyzer
try:
    sid = SentimentIntensityAnalyzer()
except LookupError:
    nltk.download("vader_lexicon")
    sid = SentimentIntensityAnalyzer()

# ──────────────────────────────────────────────────────────────────────────────
# 3. Knowledge bases  (keep your full dicts here)
# ──────────────────────────────────────────────────────────────────────────────
FEATURES_KB = {
    # Original
    "sunroof": "The power sunroof can tilt or slide open using the overhead switch.",
    "adaptive_cruise_control": "ACC maintains set speed and distance; press the steering‑wheel ACC button to activate and use +/- to adjust.",
    "lane_assist": "Lane‑Keeping Assist steers gently to keep the vehicle centered when lane markers are visible.",
    "heated_seats": "Press the seat‑icon button on the climate panel to cycle through high, medium, low, and off.",
    "parking_sensors": "Sensors emit beeps that increase in frequency as you near an obstacle – active in REVERSE or below 6 mph.",
    "apple_carplay": "Connect via USB (or wireless if equipped) and accept the CarPlay prompt on both car and phone.",
    "android_auto": "Plug in with a quality USB‑C cable or pair wirelessly, then follow on‑screen steps to launch Android Auto.",
    "blind_spot_monitor": "An amber icon lights up on the side mirror when another vehicle is in your blind spot.",
    "automatic_headlights": "Leave the stalk in AUTO; the sensor switches between DRL and low‑beam as ambient light changes.",
    "remote_start": "Press lock once, then hold the remote‑start button for 3 seconds; lights flash when the engine starts.",
    "head_up_display": "The HUD projects speed and navigation onto the windshield; adjust height in Settings › Display › HUD.",
    "360_camera": "Bird‑eye view combines images from four cameras to aid parking; press the CAMERA button to activate.",
    "wireless_charging_pad": "Place a Qi‑enabled phone on the pad; the LED turns orange while charging, green when full.",
    "ambient_lighting": "Change cabin accent colors in Settings › Lighting. Brightness syncs with headlight state.",
    "driver_memory_seat": "Hold SET then press 1 or 2 after adjusting seat, mirrors, and HUD to store your profile.",
    "power_tailgate": "Press the dash button or key‑fob twice to open/close the tailgate electrically.",
    "hands_free_liftgate": "With key in pocket, kick under the rear bumper to open the liftgate hands‑free.",
    "rain_sensing_wipers": "Wipers auto‑activate when moisture is detected; adjust sensitivity with the stalk ring.",
    "auto_dimming_mirror": "Mirror darkens automatically against glare; toggle the rear switch to disable.",
    "traffic_sign_recognition": "Front camera reads speed‑limit signs and shows them in the cluster display.",
    "automatic_emergency_braking": "AEB brakes automatically to avoid collisions below ~30 mph; keep radar area clear of debris.",
    "forward_collision_warning": "FCW gives audible & visual alerts if closing too quickly on a vehicle ahead.",
    "rear_cross_traffic_alert": "While reversing, RCTA warns of vehicles approaching from the sides.",
    "hill_descent_control": "HDC maintains low speed on steep downhill grades; press the HDC button while in 4‑Low.",
    "hill_start_assist": "HSA holds brake pressure for 2 seconds on hills to prevent rollback when starting.",
    "drive_mode_selector": "Rotate the console dial to choose Eco, Comfort, Sport, Snow, or Off‑Road modes.",
    "sport_mode": "Sport mode sharpens throttle response and holds gears longer for spirited driving.",
    "eco_mode": "Eco mode softens acceleration and optimizes A/C to maximise fuel economy.",
    "air_ionizer": "Cabin air ionizer reduces particulates and odors when RECIRC is active.",
    "rear_seat_reminder": "An alert sounds if rear doors opened at start of trip but not reopened when you stop.",
    "child_safety_lock": "Activate electronic child locks via Settings › Doors; an icon confirms engagement.",
    "stop_start_system": "Engine auto‑stops at traffic lights; depress the brake firmly. Disable via the A‑OFF button.",
    "adaptive_headlights": "Headlights swivel with steering angle to illuminate curves.",
    "matrix_led_headlights": "Matrix LEDs mask out oncoming traffic while maintaining high beam elsewhere.",
    "night_vision_assist": "Infra‑red camera highlights pedestrians/animals beyond headlight range on the cluster screen.",
    "adaptive_damping": "Electronic dampers adjust firmness every 10 ms based on road & drive mode.",
    "air_suspension": "Air suspension raises for off‑road (+40 mm) or lowers at highway speed (‑15 mm).",
    "active_noise_cancellation": "Cabin microphones detect road noise; speakers emit counter‑phase sound to cancel it.",
    "gesture_control": "Wave right‑hand gestures near the console sensor to change tracks or adjust volume.",
    "voice_control": "Say 'Hey Car' followed by a command to set navigation, climate, or seat heat.",
    "wifi_hotspot": "Enable Wi‑Fi Hotspot in Connectivity menu; supports up to 8 devices on 4G LTE.",
    "over_the_air_updates": "Vehicle software downloads overnight via Wi‑Fi; schedule install under Settings › Software.",
    "digital_instrument_cluster": "12.3‑inch cluster offers Classic, Sport, and Map‑View layouts selectable with the ◄► buttons.",
    "remote_parking": "Use the key‑fob arrows to move the car straight in/out of tight spaces remotely.",
    "valet_mode": "Valet Mode limits speed to 55 mph and hides personal data; set a PIN to activate.",
    "battery_preconditioning": "In EV mode, schedule pre‑conditioning so the battery warms before fast‑charging.",
    "plug_in_charge_port": "The charge port is on the driver‑side rear; press the door to open or use the key.",
    "regen_braking_strength": "Tap the steering paddles to cycle regenerative braking between Levels 0‑3.",
    "supercharger_locator": "The nav automatically suggests Superchargers on long trips; sites show power availability.",
    "summon_feature": "From the app, use Smart Summon to have the car drive to you within line‑of‑sight.",
}

TROUBLE_KB = {
    "engine_wont_start": "Check that the key fob battery isn't depleted, the gear selector is in PARK, and the brake pedal is fully depressed while pressing START.",
    "bluetooth_not_connecting": "Delete the car from your phone's Bluetooth list, then redo the pairing process. Ensure no more than 5 devices are stored in the infotainment system.",
    "ac_not_cold": "Verify that the A/C button is illuminated. If so, inspect cabin air filter (behind the glovebox) for clogging. Low refrigerant may require service.",
    "ac_warm_air": "Verify that the A/C button is illuminated. If so, inspect cabin air filter (behind the glovebox) for clogging. Low refrigerant may require service.",
    "ac_blows_warm": "Verify that the A/C button is illuminated. If so, inspect cabin air filter (behind the glovebox) for clogging. Low refrigerant may require service.",
}
MANUAL_KB = {
    # Original 10
    "tire_pressure": "Cold tire pressure: 35 psi front / 33 psi rear.",
    "oil_type": "Use 0W‑20 synthetic oil (API SP). Capacity 4.7 qt with filter.",
    "fuse_box_location": "Cabin fuse panel is behind driver‑lower dash. Spare fuses included.",
    "maintenance_schedule": "Oil 10 k mi; cabin filter 15 k mi; brake fluid 24 mo; coolant 60 k mi.",
    "wiper_blade_size": "26 in driver, 18 in passenger, 12 in rear.",
    "coolant_type": "HOAT 50/50 premix ethylene‑glycol. Capacity 8.2 L.",
    "spark_plug_gap": "0.044 in (1.1 mm) iridium plugs.",
    "battery_location": "12 V battery: under‑hood, passenger‑side rear corner.",
    "towing_capacity": "Max braked: 3 500 lb (with trailer brakes).",
    "fuel_tank_capacity": "Fuel tank holds 14.5 US gallons including 2 gal reserve.",
    "brake_pad_min_thickness": "Replace brake pads when friction material is ≤3 mm.",
    "transmission_fluid_type": "Use ATF WS; service interval 60 k mi.",
    "recommended_octane": "Use unleaded gasoline 91 AKI (95 RON) or higher for optimal performance.",
    "engine_air_filter_part": "Engine air filter element: OEM #17801‑0H050.",
    "cabin_filter_part": "Cabin filter: OEM #87139‑02020 activated‑carbon type.",
    "lug_nut_torque": "Torque alloy‑wheel lug nuts to 85 lb‑ft (115 N·m).",
    "serpentine_belt_routing": "See label under hood for belt path; tensioner located passenger‑side.",
    "vin_location": "VIN is stamped on driver‑side dash, visible through windshield, and on door‑jamb label.",
    "jack_points": "Use reinforced pinch‑welds behind front wheels and ahead of rear wheels.",
    "child_seat_anchors": "Lower anchors located at the seat bight; top‑tethers on rear shelf.",
    "tire_rotation_pattern": "For FWD, move front to rear same side; rear to front opposite sides every 5 k mi.",
    "wheel_alignment_spec": "Front camber −0°45′ ± 0°30′, toe 0.00 ± 0.05 in.",
    "idle_rpm": "Normal warm‑idle speed: 650 ± 50 RPM.",
    "headlight_bulb_type": "Low‑beam: H11 55 W halogen; High‑beam: 9005 60 W.",
    "obd_port_location": "OBD‑II connector is under the steering column, left of the brake pedal.",
    "reset_service_light": "Turn ignition to ON, hold trip button, power OFF, keep holding, power ON until light resets.",
    "boot_space": "Cargo capacity: 17.2 cu ft (487 L) with rear seats up; 54 cu ft (1 530 L) folded.",
    "trunk_capacity": "Cargo capacity: 17.2 cu ft (487 L) with rear seats up; 54 cu ft (1 530 L) folded.",

}
# ──────────────────────────────────────────────────────────────────────────────
# 4. Advanced normaliser
# ──────────────────────────────────────────────────────────────────────────────
_SYNONYMS = (
    # A/C
    ("a/c","ac"), ("air-con","ac"), ("air con","ac"),
    ("air-conditioning","ac"), ("air conditioning","ac"),
    # UK ↔ US parts
    ("tyre","tire"), ("tyres","tires"),
    ("bonnet","hood"), ("boot","trunk"),
    ("windscreen","windshield"), ("indicator","turn signal"),
    ("petrol","gas"), ("accelerator","gas pedal"),
    # HUD & lane-keeping
    ("head-up display","hud"), ("head up display","hud"),
    ("projector","hud"), ("dashboard projection","hud"),
    ("lane-keeping","lane keeping"), ("lane assist","lane keeping system"),
    # misc
    ("blu-tooth","bluetooth"), ("head lights","headlights"), ("head light","headlight")
)
def _normalize(txt:str)->str:
    txt = unicodedata.normalize("NFKD",txt).encode("ascii","ignore").decode().lower()
    for old,new in _SYNONYMS: txt = txt.replace(old,new)
    txt = re.sub(r"[^\w\s]"," ",txt)
    return " ".join(txt.split())

# ── 5. OpenAI embeddings (lower threshold) ───────────────────────────────────
_EMBED_MODEL = "text-embedding-3-small"
_KB_EMBEDS:dict[str,np.ndarray]|None = None
def _build_kb_embeds():
    global _KB_EMBEDS
    if _KB_EMBEDS is not None or not os.getenv("OPENAI_API_KEY"): return
    openai.api_key = os.getenv("OPENAI_API_KEY")
    keys = list(FEATURES_KB)+list(TROUBLE_KB)+list(MANUAL_KB)
    for i in range(0,len(keys),96):
        chunk = keys[i:i+96]
        rsp = openai.embeddings.create(model=_EMBED_MODEL, input=chunk)
        if _KB_EMBEDS is None: _KB_EMBEDS={}
        for k,row in zip(chunk,rsp.data):
            _KB_EMBEDS[k] = np.asarray(row.embedding,dtype="float32")

def _embed(text:str)->np.ndarray|None:
    if not os.getenv("OPENAI_API_KEY"): return None
    openai.api_key = os.getenv("OPENAI_API_KEY")
    try:
        rsp = openai.embeddings.create(model=_EMBED_MODEL, input=[text])
        return np.asarray(rsp.data[0].embedding,dtype="float32")
    except Exception: return None

def _semantic_match(text:str, thr:float=0.75) -> str|None:   # ← threshold ↓
    _build_kb_embeds()
    if _KB_EMBEDS is None: return None
    q = _embed(text)
    if q is None: return None
    best_key, best = None, 0.0
    for k,vec in _KB_EMBEDS.items():
        sim = float(np.dot(q,vec)/(np.linalg.norm(q)*np.linalg.norm(vec)))
        if sim>best: best_key,best = k,sim
    return best_key if best>=thr else None

# ── 6. GPT completion fallback (unchanged) ───────────────────────────────────
def llm_vehicle_answer(msg:str)->str|None:
    if not (key:=os.getenv("OPENAI_API_KEY")): return None
    openai.api_key = key
    try:
        prompt = ("You are an automotive virtual assistant. "
                  "Answer questions about vehicle features, troubleshooting or specs clearly and concisely.")
        r = openai.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            messages=[{"role":"system","content":prompt},
                      {"role":"user",  "content":msg}],
            max_tokens=180, temperature=0.3)
        return r.choices[0].message.content.strip()
    except Exception: return None

# ── 7. Chatbot class (fuzzy threshold 70 → 60) ───────────────────────────────
class CarChatbot:
    THRESH = 60

    @staticmethod
    def _best_match(text:str, kb:dict[str,str])->str|None:
        norm=_normalize(text)
        best_key,best= None,0
        for k in kb:
            score=fuzz.token_set_ratio(_normalize(k.replace("_"," ")), norm)
            if score>best: best_key,best = k,score
        return best_key if best>=CarChatbot.THRESH else None

    @staticmethod
    def _is_frustrated(text:str)->bool:
        return sid.polarity_scores(text)["compound"] < -0.5

    def answer(self, text:str)->str:
        # semantic first
        if (k:=_semantic_match(text)):
            if k in TROUBLE_KB: return TROUBLE_KB[k]
            if k in FEATURES_KB: return FEATURES_KB[k]
            if k in MANUAL_KB:   return MANUAL_KB[k]

        # fuzzy tiers
        for kb in (TROUBLE_KB, FEATURES_KB, MANUAL_KB):
            if (k:=self._best_match(text,kb)): return kb[k]

        # GPT fallback
        if (llm:=llm_vehicle_answer(text)): return llm

        return ("Sorry, I didn't quite get that. "
                "Could you rephrase or specify what you're looking for?")

    def chat(self):
        print("CarBot ready! Ask about features, troubleshooting, or manual info. Type 'quit' to exit.\n")
        while True:
            user = input("You: ")
            if user.strip().lower() in {"quit","exit"}:
                print("CarBot: Bye! Drive safe."); break
            reply = self.answer(user)
            if self._is_frustrated(user):
                reply += ("\nI sense some frustration. "
                          "Would you like me to connect you to a human support agent?")
            print(f"CarBot: {reply}\n")

# ── 8. Script entry-point ────────────────────────────────────────────────────
if __name__ == "__main__":
    CarChatbot().chat()

CarBot ready! Ask about features, troubleshooting, or manual info. Type 'quit' to exit.

You: what is the car boot space?
CarBot: Cargo capacity: 17.2 cu ft (487 L) with rear seats up; 54 cu ft (1 530 L) folded.

You: Awful! the ac is not working
CarBot: Verify that the A/C button is illuminated. If so, inspect cabin air filter (behind the glovebox) for clogging. Low refrigerant may require service.
I sense some frustration. Would you like me to connect you to a human support agent?

You: how do i turn the headlights to low beam ?
CarBot: Headlights swivel with steering angle to illuminate curves.

You: how to check air pressure?
CarBot: Cold tire pressure: 35 psi front / 33 psi rear.

You: what oil do i need to use ?
CarBot: Sorry, I didn't quite get that. Could you rephrase or specify what you're looking for?

You: how do i work on eco mode?
CarBot: Eco mode softens acceleration and optimizes A/C to maximise fuel economy.

You: what is the issue with heated seats?
CarBot: Press the s